<a href="https://colab.research.google.com/github/urieliram/statistical/blob/main/Tarea8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import numpy as np
import pandas as pd
import itertools
import warnings
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV, LarsCV
from sklearn.metrics import (roc_curve, roc_auc_score, confusion_matrix, accuracy_score, f1_score, precision_recall_curve) 
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from scipy.stats import bernoulli
from scipy.stats import norm
import scipy.stats
import statsmodels.api as sm
import statsmodels.tools.eval_measures as bias
import seaborn as sns
import arviz as az
import pymc3 as pm
import theano.tensor as tt
from IPython.core.pylabtools import figsize
warnings.filterwarnings("ignore")
print("Running on PyMC3 v{pm.__version__}")

Running on PyMC3 v{pm.__version__}


#Inferencia Bayesiana
queremos saber las distribuciones de probabilidad de los parámetros desconocidos del modelo.

Probar que tan buenos son unos parámetros. Cuanto mayor sea la probabilidad P(teta|x) de los valores de los parámetros dados los datos, más probable será que sean los parámetros "reales" de la distribución de la población.

Esto significa que podemos transformar nuestro problema de encontrar los parámetros de la distribución de la población, a encontrar los valores de los parámetros que maximizan el valor P(θ|x).

podemos transformar nuestro problema de encontrar los parámetros de la distribución de la población, a encontrar los valores de los parámetros que maximizan el valor P(θ|x).

Leemos los datos para un ejemplo de dos variables

In [28]:
df = pd.read_csv('bones.csv')
df = df.assign(const=1)
#print(df)
dfy = df['frac'] 
dfx = df[['const','sex','age']] ## Predictors
x   = pd.DataFrame(df[['sex','age']]).to_numpy()
y   = pd.DataFrame(df[['frac']]).to_numpy()
df.sample(5)
df.describe()

,sex,age,frac,const
count,100.00,100.000000,100.000000,100.0
mean,0.55,64.980000,0.630000,1.0
std,0.50,8.671304,0.485237,0.0
min,0.00,50.000000,0.000000,1.0
25%,0.00,57.000000,0.000000,1.0
50%,1.00,66.000000,1.000000,1.0
75%,1.00,72.000000,1.000000,1.0
max,1.00,80.000000,1.000000,1.0


## Un ejemplo básico de regresión logistica bayesiana
El ejemplo usado es un modelo de regresión logística simulado de fracturas óseas con variables independientes de edad y sexo. Fuente: [Lawrence Joseph](http://www.medicine.mcgill.ca/epidemiology/Joseph/courses/EPIB-621/main.html) [PDF](http://www.medicine.mcgill.ca/epidemiology/joseph/courses/EPIB-621/bayeslogit.pdf)

Al principio no sabemos nada sobre los parámetros `beta` así que usaremos una distribución **uniforme**  con límites suficientemente grandes como los valores: `lower = -10**6; higher = 10**6 `


In [ ]:
lower = -10**6; higher = 10**6
with pm.Model() as first_model:
    ## Priors on parameters
    beta_0   = pm.Uniform('beta_0', lower=lower, upper= higher)
    beta_sex = pm.Uniform('beta_sex', lower, higher)
    beta_age = pm.Uniform('beta_age', lower, higher)
    
    #the probability of output equal to 1
    p = pm.Deterministic('p', pm.math.sigmoid(beta_0+beta_sex*df['sex']+ beta_age*df['age']))

with first_model:
    #fit the data 
    observed=pm.Bernoulli("frac", p, observed=df['frac'])
    start = pm.find_MAP()
    step  = pm.Metropolis()
    
    #samples from posterior distribution 
    trace=pm.sample(25000, step=step, start=start)
    first_burned_trace=trace[15000:]

Sequential sampling (2 chains in 1 job)
CompoundStep
>Metropolis: [beta_age]
>Metropolis: [beta_sex]
>Metropolis: [beta_0]


In [ ]:
##Graficamos las distribuciones resultantes de nuestro primer modelo
pm.traceplot(first_burned_trace, figsize=[14,14])
plt.savefig('fig_t8_1.png', transparent=True)
plt.show()

In [ ]:
## Ahora calculamos la media de nuestra primera versión de las muestras generadas por la simulación.
coeffs=['beta_0', 'beta_sex', 'beta_age']
d=dict()
for item in coeffs:
    d[item]=[first_burned_trace[item].mean()]
    
result_coeffs=pd.DataFrame.from_dict(d)    
result_coeffs
#coeff_result=pd.DataFrame(d)    
#coeff_result

Una ventaja del enfoque de inferencia bayesiana es que no solo nos da la media de los parámtros del modelo, también podemos obtener los intervalos de confianza al 95%. Es decir que los Los parámetros buscados se encontrarán entre los valores siguientes con una probabilidad del 95%. 

In [ ]:
mean = first_burned_trace['beta_0'].mean()
hpd = az.hdi(first_burned_trace['beta_0'].flatten())

coeffs=['beta_0', 'beta_sex', 'beta_age']
interval=dict()
for item in coeffs:
    interval[item]=az.hdi(first_burned_trace[item].flatten()) #compute 95% high density interval
    
result_coeffs=pd.DataFrame.from_dict(interval).rename(index={0: 'lower', 1: 'upper'})
result_coeffs

Los valores buscados se encuentran entre estos valores, ahora podemos refinar la búsqueda de los parámetros usando estos límites en un segundo modelo.

In [ ]:
with pm.Model() as second_model:
    ## Priors on parameters
    beta_0   = pm.Uniform('beta_0',   lower=-31.561240, upper= -20.376186)
    beta_sex = pm.Uniform('beta_sex', lower=0.555843,   upper=2.816436)
    beta_age = pm.Uniform('beta_age', lower=0.314423,   upper=0.487489)
    
    #the probability of output equal to 1
    p = pm.Deterministic('p', pm.math.sigmoid(beta_0+beta_sex*df['sex']+ beta_age*df['age']))
    
with second_model:
    #fit the data 
    observed=pm.Bernoulli("frac", p, observed=df['frac'])
    start = pm.find_MAP()
    step = pm.Metropolis()
    
    #samples from posterior distribution 
    trace=pm.sample(25000, step=step, start=start)
    second_burned_trace=trace[15000:]

In [ ]:
##Graficamos las distribuciones resultantes de nuestro segundo modelo
pm.traceplot(second_burned_trace, figsize=[14,14])
plt.savefig('fig_t8_2.png', transparent=True)
plt.show()

In [ ]:
## Ahora calculamos la media de nuestra primera versión de las muestras generadas por la simulación.
coeffs=['beta_0', 'beta_sex', 'beta_age']
d=dict()
for item in coeffs:
    d[item]=[second_burned_trace[item].mean()]
    
result_coeffs=pd.DataFrame.from_dict(d)    
print(result_coeffs)

In [ ]:
## Ahora calculamos intervalos al 95%.
mean = second_burned_trace['beta_0'].mean()
hpd = az.hdi(second_burned_trace['beta_0'].flatten())

coeffs=['beta_0', 'beta_sex', 'beta_age']
interval=dict()
for item in coeffs:
    interval[item]=az.hdi(second_burned_trace[item].flatten()) #compute 95% high density interval
    
result_coeffs=pd.DataFrame.from_dict(interval).rename(index={0: 'lower', 1: 'upper'})
print(result_coeffs)

Ahora, entrenemos el modelo asumiendo que los coeficientes de la regresión logistica siguen distribuciones normales. Es decir cambiaremos el conjunto de priors en un tercer modelo.

In [ ]:
with pm.Model() as third_model:  
    ## Priors on parameters
    beta_0   = pm.Normal('beta_0'  , mu=-23.764747, sd=10**4)
    beta_sex = pm.Normal('beta_sex', mu=1.572192, sd=10**4)
    beta_age = pm.Normal('beta_age', mu=0.37384, sd=10**4)
    
    #the probability of output equal to 1
    p = pm.Deterministic('p', pm.math.sigmoid(beta_0+beta_sex*df['sex']+ beta_age*df['age']))
    
with third_model:
    #fit the data 
    observed=pm.Bernoulli("frac", p, observed=df['frac'])
    start = pm.find_MAP()
    step = pm.Metropolis()
    
    #samples from posterior distribution 
    trace=pm.sample(25000, step=step, start=start)
    third_burned_trace=trace[15000:]

In [ ]:
##Graficamos las distribuciones resultantes de nuestro tercer modelo
pm.traceplot(third_burned_trace, figsize=[14,14])
plt.savefig('fig_t8_3.png', transparent=True)
plt.show()

In [ ]:
## Ahora calculamos la media de nuestra primera versión de las muestras generadas por la simulación.
coeffs=['beta_0', 'beta_sex', 'beta_age']
d=dict()
for item in coeffs:
    d[item]=[third_burned_trace[item].mean()]
    
result_coeffs=pd.DataFrame.from_dict(d)    
print(result_coeffs)

In [ ]:
## Ahora calculamos intervalos al 95%.
mean = third_burned_trace['beta_0'].mean()
hpd = az.hdi(third_burned_trace['beta_0'].flatten())

coeffs=['beta_0', 'beta_sex', 'beta_age']
interval=dict()
for item in coeffs:
    interval[item]=az.hdi(third_burned_trace[item].flatten()) #compute 95% high density interval
    
result_coeffs=pd.DataFrame.from_dict(interval).rename(index={0: 'lower', 1: 'upper'})
print(result_coeffs)

Como podemos ver los intervalos son mas cerrados a los que teníamos en el primer modelo.

## Modelo lineal generalizado (GLM) en PyMC3 en fracturas oseas.
Hemos hecho un análisis inferencial bayesiano expresando los priors de cada una de las variables. Sin embargo, cuando el número de variables es muy grande, **PyMC3** tiene un modelo lineal generalizado en el que todo esta automatizado. Se usará este modelo para ajustar nuestros datos.

In [ ]:
with pm.Model() as fourth_model:
    pm.glm.GLM.from_formula('frac ~ sex + age',df, 
                            family=pm.glm.families.Binomial())
    fourth_trace = pm.sample(25000, tune=10000, init='adapt_diag')
pm.traceplot(fourth_trace, figsize=[12,14])
plt.savefig('fig_t8_4.png', transparent=True)
plt.show()

In [ ]:
pm.summary(fourth_trace)

In [ ]:
with fourth_model:
    map_solution=pm.find_MAP()
d=dict()
for item in map_solution.keys():
    d[item]=[float(map_solution[item])]
    
fourth_map_coeffs=pd.DataFrame.from_dict(d)    
fourth_map_coeffs

In [ ]:
## Ahora calculamos la media de nuestra primera versión de las muestras generadas por la simulación.
coeffs=['age', 'sex', 'Intercept']
d=dict()
for item in coeffs:
    d[item]=[fourth_trace[item].mean()]
    
result_coeffs=pd.DataFrame.from_dict(d)    
print(result_coeffs)

In [ ]:
## Ahora calculamos intervalos al 95%.
mean = fourth_trace['Intercept'].mean()
hpd = az.hdi(fourth_trace['Intercept'].flatten())

coeffs=['age', 'sex', 'Intercept']
interval=dict()
for item in coeffs:
    interval[item]=az.hdi(fourth_trace[item].flatten()) #compute 95% high density interval
    
result_coeffs=pd.DataFrame.from_dict(interval).rename(index={0: 'lower', 1: 'upper'})
print(result_coeffs)

Por último, calculamos la matriz de confusión del ajuste al modelo de regresión logistica

In [ ]:
with fourth_model:
    ppc = pm.sample_posterior_predictive(fourth_trace, samples=15000)
#compute y_score 
with fourth_model:
    fourth_y_score = np.mean(ppc['y'], axis=0)
#convert y_score into binary decisions    
fourth_model_prediction=[1 if x >0.5 else 0 for x in fourth_y_score]
#compute confussion matrix 
fourth_model_confussion_matrix = confusion_matrix(df['frac'], fourth_model_prediction)
fourth_model_confussion_matrix

## Versión frecuentista de la regresión logística con la librería *statsmodel*.

Ahora, comparamos los resultados con un análisis frecuentista de regresión logística de la librería **statsmodels**.

In [ ]:
model = sm.Logit(dfy, dfx)
results = model.fit()
print(results.summary())

##Predicción de sobrecarga en grupos de líneas de transmisión.
En esta sección se usará inferencia bayesiana para ajustar un modelo de regresión logística a datos de violación de flujo de potencia eléctrica en grupos de líneas de transmisión, que interconectan regiones eléctricas. La variable dependientes es de naturaleza binaria con un valor de uno si la línea presenta sobrecarga y cero si no. Las variables independientes son el flujo neto máximo y mínimo en la región eléctrica en un día y se calcula como la diferencia entre la demanda menos la generación en cada región.


In [ ]:
df = pd.read_csv('overload.csv')
df = df.assign(const=1)
print(df)
dfy = df['L3'] #Se escoge le número de línea de transmisión
dfx = df[['CEN','NES','NOR','NTE','OCC','ORI','PEN','CEN_min','NES_min','NOR_min','NTE_min','OCC_min','ORI_min','PEN_min']] ## Predictors
df.sample(5)
df.describe()

In [ ]:
with pm.Model() as fourth_model:
    pm.glm.GLM.from_formula('L3 ~ CEN +  NES + NOR + NTE + OCC + ORI +  PEN + CEN_min + NES_min + NOR_min + NTE_min + OCC_min + ORI_min + PEN_min',df, 
                            family=pm.glm.families.Binomial())
    fourth_trace = pm.sample(25000, tune=10000, init='adapt_diag')
#https://pymc3-testing.readthedocs.io/en/rtd-docs/api/plots.html
#https://python.hotexamples.com/examples/pymc3/-/traceplot/python-traceplot-function-examples.html

pm.traceplot(fourth_trace, figsize=[16,70]) 
plt.savefig('fig_t8_5.png', transparent=True)
plt.show()

In [ ]:
pm.summary(fourth_trace)

In [ ]:
with fourth_model:
    map_solution=pm.find_MAP()
d=dict()
for item in map_solution.keys():
    d[item]=[float(map_solution[item])]
    
fourth_map_coeffs=pd.DataFrame.from_dict(d)    
fourth_map_coeffs

In [ ]:
## Ahora calculamos la media de de las muestras generadas por la simulación.
coeffs=['CEN','NES','NOR','NTE','OCC','ORI','PEN','CEN_min','NES_min','NOR_min','NTE_min','OCC_min','ORI_min','PEN_min','Intercept']
d=dict()
for item in coeffs:
    d[item]=[fourth_trace[item].mean()]
    
result_coeffs=pd.DataFrame.from_dict(d)    
print(result_coeffs)

In [ ]:
## Ahora calculamos intervalos al 95%.
mean = fourth_trace['Intercept'].mean()
hpd = az.hdi(fourth_trace['Intercept'].flatten())

coeffs=['CEN','NES','NOR','NTE','OCC','ORI','PEN','CEN_min','NES_min','NOR_min','NTE_min','OCC_min','ORI_min','PEN_min','Intercept']
interval=dict()
for item in coeffs:
    interval[item]=az.hdi(fourth_trace[item].flatten()) #compute 95% high density interval
    
result_coeffs=pd.DataFrame.from_dict(interval).rename(index={0: 'lower', 1: 'upper'})
print(result_coeffs)

Por último, calculamos la matriz de confusión del ajuste al modelo de regresión logistica

In [ ]:
with fourth_model:
    ppc = pm.sample_posterior_predictive(fourth_trace, samples=15000)
#compute y_score 
with fourth_model:
    fourth_y_score = np.mean(ppc['y'], axis=0)
#convert y_score into binary decisions    
fourth_model_prediction=[1 if x >0.5 else 0 for x in fourth_y_score]
#compute confussion matrix 
fourth_model_confussion_matrix = confusion_matrix(df['L3'], fourth_model_prediction)
fourth_model_confussion_matrix

## Versión frecuentista de la regresión logística con la librería *statsmodel*.

Ahora, comparamos los resultados con un análisis frecuentista de regresión logística de la librería **statsmodels**.

In [ ]:
model = sm.Logit(dfy, dfx)
results = model.fit()
print(results.summary())

### **Conclusión:** 
Hemos utilizado **PyMC3** para implementar la regresión logistica bayesiana para varias variables, además de la función **Logit** de la librería *statsmodel*, que implementa un enfoque frecuentista.
Los resultados de estimación de parámetros entre el enfoque frecuentista y el bayesiano son muy parecidos, sin embargo, el enfoque bayesiano da algunas ventajas ya que da la posibilidad de actualizar el modelo con nueva información, mientras que los modelos de regresión lineal generan valores únicos de los parámetros de ajuste, mientras que los modelos de regresión lineal bayesianos pueden generar distribuciones de los parámetros.
Esto tiene la ventaja de que podemos cuantificar la incertidumbre de nuestra estimación.
Otra cosa que observamos es que a pesar de que los modelos modelos bayesianos que usamos usan distribuciones prior diferentes, los rendimientos de predicción son similares. Esto quiere decir que a medida que crece el conjunto de datos los resultados deberían converger en la misma solución.

[Bayesian logistic regression with PyMC3](https://towardsdatascience.com/bayesian-logistic-regression-with-pymc3-8e17c576f31a)

[Bayesian Linear Regression in Python via PyMC3](https://towardsdatascience.com/bayesian-linear-regression-in-python-via-pymc3-ab8c2c498211)

[PyMC3](https://docs.pymc.io/en/v3/nb_examples/index.html)

[Logistic regression with PyMC3](https://goldinlocks.github.io/Bayesian-logistic-regression-with-pymc3/)

[Bayesian statistics by Lawrence Joseph (Medicine)](http://www.medicine.mcgill.ca/epidemiology/Joseph/courses/EPIB-621/main.html) [PDF](http://www.medicine.mcgill.ca/epidemiology/joseph/courses/EPIB-621/bayeslogit.pdf)

[Bayesian Learning for Machine Learning: Part II - Linear Regression](https://wso2.com/blog/research/part-two-linear-regression/)

[An Introduction to Logistic Regression in Python](https://www.simplilearn.com/tutorials/machine-learning-tutorial/logistic-regression-in-python)

[Bayesian inference tutorial: a hello world example](https://datapythonista.me/blog/bayesian-inference-tutorial-a-hello-world-example.html)

[Ejemplo de Bayes con covid y síntomas](https://statsthinking21.github.io/statsthinking21-python/10-BayesianStatistics.html)

[Estimating Probabilities with Bayesian Modeling in Python](https://towardsdatascience.com/estimating-probabilities-with-bayesian-modeling-in-python-7144be007815)

[Estimando probabilidades con inferencia bayesiana](https://github.com/WillKoehrsen/probabilistic-programming/blob/master/Estimating%20Probabilities%20with%20Bayesian%20Inference.ipynb)

Bayesian Linear Regression in Python: Using Machine Learning to Predict Student Grades: [Parte 1](https://towardsdatascience.com/bayesian-linear-regression-in-python-using-machine-learning-to-predict-student-grades-part-1-7d0ad817fca5) y [Parte2](https://towardsdatascience.com/bayesian-linear-regression-in-python-using-machine-learning-to-predict-student-grades-part-2-b72059a8ac7e)

[Bayesian Inference in Python](https://towardsai.net/p/l/bayesian-inference-in-python) 

[Bayesian Statistics explained to Beginners in Simple English](Bayesian Statistics explained to Beginners in Simple English)

[Bayesian inference in Python-coursera](https://www.coursera.org/lecture/advanced-machine-learning-signal-processing/bayesian-inference-in-python-9D2Pz)

[A Guide to Bayesian Statistics in Python for Beginners](https://analyticsindiamag.com/a-guide-to-bayesian-statistics-in-python-for-beginners/)

[BBN: Bayesian Belief Networks — How to Build Them Effectively in Python](https://towardsdatascience.com/bbn-bayesian-belief-networks-how-to-build-them-effectively-in-python-6b7f93435bba)

[Bayesian multivariate linear regression Yni](https://en.wikipedia.org/wiki/Bayesian_multivariate_linear_regression#:~:text=In%20statistics%2C%20Bayesian%20multivariate%20linear,a%20single%20scalar%20random%20variable.)

